# Compute residual norm coefficients

This notebook computes the residual norm coefficients as part of the variable weights.

In [1]:
import os
import yaml
import copy
import numpy as np
import xarray as xr

In [2]:
from scipy.stats import gmean

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

## Main routine

In [7]:
# get variable information from data_preprocessing/config
config_name = os.path.realpath('data_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)


In [ ]:
# get level info
N_levels = 13
base_dir = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_1deg/'
ds_example = xr.open_zarr(base_dir+'upper_subset/ERA5_subset_1deg_6h_1979_conserve.zarr')
level = np.array(ds_example['level'])

In [ ]:
# get variable names
varnames = list(conf['residual'].keys())
varnames = varnames[:-5] # remove save_loc and others

varname_surf = list(set(varnames) - set(['U', 'V', 'T', 'Q', 'Z', 'specific_total_water']))
varname_upper = ['U', 'V', 'T', 'Q', 'Z', 'specific_total_water']

In [8]:
# collect computed mean and variance values
# See "qsub_STEP01_compute_mean_std.ipynb"
MEAN_values = {}
STD_values = {}

for varname in varname_surf:
    save_name = conf['residual']['save_loc'] + '{}_mean_std_{}.npy'.format(
        conf['residual']['prefix'], varname)
    mean_std = np.load(save_name)
    MEAN_values[varname] = mean_std[0]
    STD_values[varname] = mean_std[1]

for varname in varname_upper:

    # -------------------------------------------- #
    # allocate all levels
    mean_std_all_levels = np.empty((2, N_levels))
    mean_std_all_levels[...] = np.nan
    
    for i_level in range(N_levels):
        save_name = conf['residual']['save_loc'] + '{}_level{}_mean_std_{}.npy'.format(
            conf['residual']['prefix'], i_level, varname)
        mean_std = np.load(save_name)
        mean_std_all_levels[:, i_level] = mean_std

    # -------------------------------------------- #
    # save
    MEAN_values[varname] = np.copy(mean_std_all_levels[0, :])
    STD_values[varname] = np.copy(mean_std_all_levels[1, :])

keys_to_drop = ['TCC', 'SKT', 'SP', 'Q', 'land_sea_CI_mask'] # <---------------- some variables are not used in the paper
MEAN_values = {k: v for k, v in MEAN_values.items() if k not in keys_to_drop}
STD_values = {k: v for k, v in STD_values.items() if k not in keys_to_drop}

In [9]:
# separate upper air (list) and surf (float) std values
std_val_all = list(STD_values.values())
std_val_surf = np.array(std_val_all[:-5])
std_val_upper = std_val_all[-5:]

# combine
std_concat = np.concatenate([std_val_surf]+ std_val_upper)

# geometrical mean (not used)
std_g = gmean(np.sqrt(std_concat))

### Save residual coef as a file

In [11]:
# ------------------------------------------------------- #
# create xr.DataArray for std
ds_std_6h = xr.Dataset(coords={"level": level})

for varname, data in STD_values.items():
    data = np.sqrt(data) # / std_g # <--- var to std and divided by std_g
    if len(data.shape) == 1:
        data_array = xr.DataArray(
            data,
            dims=["level",],
            coords={"level": level},
            name=varname,
        )
        ds_std_6h[varname] = data_array
    else:
        data_array = xr.DataArray(
            data,
            name=varname,
        )
        ds_std_6h[varname] = data_array

In [12]:
# ds_std_6h.to_netcdf(base_dir+'mean_std/residual_original_6h_1979_2019_1deg_project2.nc')

### Comparing with the old residual norm

In [11]:
# ------------------------------------------------------- #
# Compare with my old ones
std_new = xr.open_dataset(base_dir+'mean_std/residual_6h_1979_2019_1deg_project2.nc')
std_conserve = xr.open_dataset(base_dir+'mean_std/residual_6h_1979_2019_conserve_1deg.nc')

for varname in list(std_conserve.keys()):
    try:
        print('=============== {} ================='.format(varname))
        print(np.array(std_new[varname]))
        print(np.array(std_conserve[varname]))
    except:
        pass

=============== SKT =================
=============== surface_latent_heat_flux =================
2.306996458279529
2.756990226781677
=============== top_net_solar_radiation =================
5.259948532947676
6.285933663515951
=============== VAR_2T =================
0.48636395589003817
0.5812322200302392
=============== surface_net_thermal_radiation =================
2.603373302474922
3.1111771870427525
=============== toa_incident_solar_radiation =================
5.274119616967553
6.302868904142561
=============== surface_sensible_heat_flux =================
4.24858429119819
5.077296641030351
=============== MSL =================
0.8125157492432455
0.9710019153823931
=============== TCC =================
=============== total_precipitation =================
3.69710900263375
4.418252724674436
=============== top_net_thermal_radiation =================
1.5778630952817745
1.8856349420386707
=============== evaporation =================
2.3033441023752
2.752625456520836
=============== 